In [0]:
%sql

use catalog emrcatalog;

create schema if not exists gold;

CREATE TABLE IF NOT EXISTS gold.dim_department
(
Dept_Id string,
SRC_Dept_Id string,
Name string,
datasource string
);

truncate TABLE gold.dim_department;

insert into gold.dim_department
select 
distinct
Dept_Id ,
Source_Dept_Id,
Name,
datasource 
from silver.departments
where is_quarantine=false;

select * from gold.dim_department limit 10;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.dim_patient
(
    patient_key STRING,
    src_patientid STRING,
    firstname STRING,
    lastname STRING,
    middlename STRING,
    ssn STRING,
    phonenumber STRING,
    gender STRING,
    dob DATE,
    address STRING,
    datasource STRING
);

truncate TABLE gold.dim_patient;

insert into gold.dim_patient
select 
    patient_key ,
    src_patient_id ,
    firstname ,
    lastname ,
    middlename ,
    ssn ,
    phonenumber ,
    gender ,
    dob ,
    address ,
    datasource 
 from silver.patients
 where is_current=true and is_quarantined=false;

 select * from gold.dim_patient limit 10;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.dim_provider
(
ProviderID string,
FirstName string,
LastName string,
DeptID string,
NPI long,
datasource string
);

truncate TABLE gold.dim_provider;

insert into gold.dim_provider
select 
ProviderID ,
FirstName ,
LastName ,
concat(DeptID,'-',datasource) deptid,
NPI ,
datasource 
 from silver.providers
 where is_quarantine=false;

select * from gold.dim_provider limit 10;

In [0]:
%sql
create table if not exists gold.fact_transactions
(
  TransactionID string,
  SRC_TransactionID string,
  EncounterID string,
  FK_PatientID string,
  FK_ProviderID string,
  FK_DeptID string,
  ICDCode string,
  ProcedureCode string,
  VisitType string,
  ServiceDate date,
  PaidDate date,
  Amount double,
  PaidAmount double,
  AmountType string,
  ClaimID string,
  datasource string,
  refreshed_at timestamp
);

truncate table gold.fact_transactions;

insert into gold.fact_transactions
select 
  t.TransactionID, 
  t.SRC_TransactionID,
  t.EncounterID,
  concat(t.PatientID,'-',t.datasource ) as FK_Patient_ID,
  case when t.datasource='hos-a' then concat('H1-',t.providerID) else concat('H2-',t.providerID ) end as FK_Provider_ID, 
  concat(t.DeptID,'-',t.datasource ) as FK_Dept_ID, 
  t.ICDCode,
  t.ProcedureCode CPT_Code,
  t.VisitType,
  t.ServiceDate, 
  t.PaidDate,
  t.Amount Charge_Amt, 
  t.PaidAmount Paid_Amt, 
  t.AmountType,
  t.ClaimID,
  t.datasource,
  current_timestamp()
  from silver.transactions t 
  where t.is_current=true and t.is_quarantined=false;

select * from gold.fact_transactions limit 10;